In [0]:
import tensorflow as tf
from tensorflow import keras

import functools
import numpy as np

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [0]:
np.set_printoptions(3, suppress=True)

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [0]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key, value.numpy()))

In [8]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [50. 17. 27. 28. 34.]
n_siblings_spouses  : [1 1 0 0 0]
parch               : [0 0 0 0 0]
fare                : [106.425 108.9     7.896   7.25   26.55 ]
class               : [b'First' b'First' b'Third' b'Third' b'First']
deck                : [b'C' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'y']


In [9]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'female' b'male' b'male']
age                 : [22. 19. 28. 45.  3.]
n_siblings_spouses  : [0 0 0 0 1]
parch               : [0 0 0 0 1]
fare                : [10.517 10.171  7.75   8.05  26.   ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'F']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']


In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 28. 28.  4. 34.]
n_siblings_spouses  : [0 0 0 4 0]
class               : [b'Third' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'y' b'n' b'n']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS,
                           column_defaults=DEFAULTS)

show_batch(temp_dataset)

age                 : [27. 22. 16. 16. 28.]
n_siblings_spouses  : [0. 0. 1. 0. 0.]
parch               : [0. 0. 3. 0. 0.]
fare                : [ 7.925  8.05  34.375  9.217  7.879]


In [0]:
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [14]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[21.     0.     0.     7.925]
 [20.     1.     1.    15.742]
 [63.     0.     0.     9.587]
 [28.     0.     0.     7.896]
 [21.     0.     0.    77.958]]

[0 1 1 0 1]


In [15]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [38.  28.  28.  20.5 28.5]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [71.283 13.    29.7    7.25   7.229]
class               : [b'First' b'Second' b'First' b'Third' b'Third']
deck                : [b'C' b'unknown' b'C' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [0]:
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names
  
  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [19]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'female']
class               : [b'Third' b'Third' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'n' b'y' b'n' b'y' b'n']
numeric             : [[28.     3.     1.    25.467]
 [28.     0.     0.     8.05 ]
 [25.     1.     0.    26.   ]
 [29.     0.     0.     7.775]
 [28.     2.     0.    23.25 ]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data))

In [21]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [24]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f75e0378ea0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [25]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[22.   ,  0.   ,  0.   ,  7.796],
       [20.   ,  0.   ,  0.   ,  7.854],
       [26.   ,  0.   ,  0.   ,  8.05 ],
       [30.   ,  0.   ,  0.   , 31.   ],
       [ 2.   ,  3.   ,  1.   , 21.075]], dtype=float32)>

In [26]:
numeric_layer = keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.61 , -0.474, -0.479, -0.487],
       [-0.77 , -0.474, -0.479, -0.486],
       [-0.29 , -0.474, -0.479, -0.482],
       [ 0.029, -0.474, -0.479, -0.062],
       [-2.208,  2.132,  0.782, -0.244]], dtype=float32)

In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
      key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [0]:
preprocessing_layer = keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [33]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.61  -0.474
 -0.479 -0.487  1.     0.   ]


In [0]:
model = keras.Sequential([
  preprocessing_layer,
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(1),                          
])

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [36]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 4ms/step - loss: 0.5094 - accuracy: 0.7337
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4272 - accuracy: 0.8166
Epoch 3/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3987 - accuracy: 0.8214
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3960 - accuracy: 0.8357
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3868 - accuracy: 0.8341
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3760 - accuracy: 0.8389
Epoch 7/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3712 - accuracy: 0.8293
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3656 - accuracy: 0.8437
Epoch 9/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3618 - accuracy: 0.8389
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3504 - accuracy: 0.8453

In [37]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 3ms/step - loss: 0.4449 - accuracy: 0.8409


Test loss 0.4448831081390381, Test Accuracy 0.8409090638160706


In [38]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 5.39%  | Actual outcome:  DIED
Predicted survival: 12.27%  | Actual outcome:  SURVIVED
Predicted survival: 6.95%  | Actual outcome:  DIED
Predicted survival: 77.06%  | Actual outcome:  DIED
Predicted survival: 7.16%  | Actual outcome:  DIED
